# Movies ratings analysis with a small dataset (100K entries)

### At first we will explore this dataset with pandas then we will compare computation performances between pandas and pyspark

In [1]:
import pandas as pd
from pyarrow import csv, compute
import pyarrow as pa
import time
from datetime import datetime

In [2]:
movies_file_small = "./small_dataset/movies.csv"
ratings_file_small = "./small_dataset/ratings.csv"
movies_file_large = "./large_dataset/movies.csv"
ratings_file_large = "./large_dataset/ratings.csv"

In [3]:
movies_df = pd.read_csv(movies_file_small)
ratings_df = pd.read_csv(ratings_file_small)

### Print dataframes to see the data structure

In [4]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## Exploration with pandas

### Question 1 : how many adventure movies are there ?

In [6]:
adventure_movies_nb = movies_df.loc[movies_df["genres"].str.contains("Adventure")].shape[0]
print(f"There are {adventure_movies_nb} movies in category Adventure")

There are 1263 movies in category Adventure


### Question 2 : How many movies by category are there ?

In [7]:
movies_by_category = {}
for movie in movies_df.iterrows():
    categories = movie[1]["genres"].split("|")
    for category in categories:
        if movies_by_category.get(category) is None:
            movies_by_category[category] = 1
        else:
            movies_by_category[category] += 1
for i in movies_by_category:
    print(f"{i}: {movies_by_category[i]}")

Adventure: 1263
Animation: 611
Children: 664
Comedy: 3756
Fantasy: 779
Romance: 1596
Drama: 4361
Action: 1828
Crime: 1199
Thriller: 1894
Horror: 978
Mystery: 573
Sci-Fi: 980
War: 382
Musical: 334
Documentary: 440
IMAX: 158
Western: 167
Film-Noir: 87
(no genres listed): 34


### Question 3 : calculate mean of ratings by user (+ standard deviation and variance)

In [8]:
user_ratings_mean = ratings_df[["userId", "rating"]].groupby(["userId"]).mean()
user_ratings_mean

,rating
userId,
1,4.366379
2,3.948276
3,2.435897
4,3.555556
5,3.636364
...,...
606,3.657399
607,3.786096
608,3.134176


In [9]:
# With standard deviation and variance
user_ratings_stats = ratings_df[["userId", "rating"]].groupby(["userId"]).aggregate(["mean", "std", "var"])
user_ratings_stats

rating                    
            mean       std       var
userId                              
1       4.366379  0.800048  0.640077
2       3.948276  0.805615  0.649015
3       2.435897  2.090642  4.370783
4       3.555556  1.314204  1.727132
5       3.636364  0.990441  0.980973
...          ...       ...       ...
606     3.657399  0.724121  0.524351
607     3.786096  0.965657  0.932494
608     3.134176  1.079262  1.164807
609     3.270270  0.450225  0.202703
610     3.688556  0.857422  0.735173

[610 rows x 3 columns]

### Question 4 : Calculate score of each movie

In [10]:
# at first, let's transform timestamps column into dates
ratings_df["timestamp"] = ratings_df["timestamp"].apply(lambda x: datetime.fromtimestamp(x))
ratings_df = ratings_df.rename(columns={"timestamp": "date"})

# next step, we need to join the 2 dataframes
df = pd.merge(ratings_df, movies_df, on="movieId")
df

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 20:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,1996-11-08 07:36:02,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,2005-01-25 07:52:26,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,2017-11-13 13:59:30,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,2011-05-18 07:28:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,2016-11-19 09:55:49,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,2016-11-19 09:43:18,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,2017-05-03 22:53:14,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,2017-05-03 23:59:49,Blair Witch (2016),Horror|Thriller


In [11]:
# a simple score for each movie could be juste the mean of all ratings for each of them
movie_rating = df.groupby("movieId").aggregate({"rating": ["mean"]})
movie_rating

,rating
,mean
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000


In [12]:
# now let's merge movies_df with the score
movies_df = pd.merge(movies_df, movie_rating, on="movieId")
movies_df = movies_df.rename(columns={("rating", "mean"): "score"})
movies_df

/tmp/ipykernel_70109/588562798.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  movies_df = pd.merge(movies_df, movie_rating, on="movieId")


,movieId,title,genres,score
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429
...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.000000
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.500000
9721,193585,Flint (2017),Drama,3.500000
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.500000


# Now let's do all of this with pandas and calculate computation time

In [13]:
def pandas_compute(movies_file, ratings_file):
    # get dataframes
    movies_df = pd.read_csv(movies_file)
    ratings_df = pd.read_csv(ratings_file)
    
    # question 1 : number of movies in category Adventure
    adventure_movies_nb = movies_df.loc[movies_df["genres"].str.contains("Adventure")].shape[0]
    print(f"There are {adventure_movies_nb} movies in category Adventure")
    
    # question 2 : number of movies by categories
    movies_by_category = {}
    for movie in movies_df.iterrows():
        categories = movie[1]["genres"].split("|")
        for category in categories:
            if movies_by_category.get(category) is None:
                movies_by_category[category] = 1
            else:
                movies_by_category[category] += 1
    for i in movies_by_category:
        print(f"{i}: {movies_by_category[i]}")
    
    # question 3 : mean of ratings per user
    user_ratings_stats = ratings_df[["userId", "rating"]].groupby(["userId"]).aggregate(["mean", "std", "var"])
    print(user_ratings_stats)
    
    # question 4 : score for each movie
    ratings_df["timestamp"] = ratings_df["timestamp"].apply(lambda x: datetime.fromtimestamp(x))
    ratings_df = ratings_df.rename(columns={"timestamp": "date"})
    df = pd.merge(ratings_df, movies_df, on="movieId")
    movie_rating = df.groupby("movieId").aggregate({"rating": ["mean"]})
    movies_df = pd.merge(movies_df, movie_rating, on="movieId")
    movies_df = movies_df.rename(columns={("rating", "mean"): "score"})
    print(movies_df)

In [14]:
%%time
pandas_compute(movies_file_small, ratings_file_small)

There are 1263 movies in category Adventure
Adventure: 1263
Animation: 611
Children: 664
Comedy: 3756
Fantasy: 779
Romance: 1596
Drama: 4361
Action: 1828
Crime: 1199
Thriller: 1894
Horror: 978
Mystery: 573
Sci-Fi: 980
War: 382
Musical: 334
Documentary: 440
IMAX: 158
Western: 167
Film-Noir: 87
(no genres listed): 34
          rating                    
            mean       std       var
userId                              
1       4.366379  0.800048  0.640077
2       3.948276  0.805615  0.649015
3       2.435897  2.090642  4.370783
4       3.555556  1.314204  1.727132
5       3.636364  0.990441  0.980973
...          ...       ...       ...
606     3.657399  0.724121  0.524351
607     3.786096  0.965657  0.932494
608     3.134176  1.079262  1.164807
609     3.270270  0.450225  0.202703
610     3.688556  0.857422  0.735173

[610 rows x 3 columns]
      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2        

/tmp/ipykernel_70109/3622626335.py:31: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  movies_df = pd.merge(movies_df, movie_rating, on="movieId")


In [15]:
%%time
pandas_compute(movies_file_large, ratings_file_large)

There are 4067 movies in category Adventure
Adventure: 4067
Animation: 2663
Children: 2749
Comedy: 15956
Fantasy: 2637
Romance: 7412
Drama: 24144
Action: 7130
Crime: 5105
Thriller: 8216
Horror: 5555
Mystery: 2773
Sci-Fi: 3444
IMAX: 197
Documentary: 5118
War: 1820
Musical: 1113
Western: 1378
Film-Noir: 364
(no genres listed): 4266
          rating                    
            mean       std       var
userId                              
1       3.312500  1.046821  1.095833
2       3.666667  0.487950  0.238095
3       3.545455  0.820200  0.672727
4       3.397418  1.231395  1.516333
5       4.263889  0.581234  0.337833
...          ...       ...       ...
283224  3.732523  0.894533  0.800189
283225  3.100000  0.640723  0.410526
283226  1.818182  0.603023  0.363636
283227  3.941176  0.845620  0.715074
283228  4.340708  0.648713  0.420828

[283228 rows x 3 columns]
       movieId                                    title  \
0            1                         Toy Story (1995)   
1    

/tmp/ipykernel_70109/3622626335.py:31: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  movies_df = pd.merge(movies_df, movie_rating, on="movieId")


# So we have 360ms for the small dataset and 21s for the large dataset with pandas